In [1]:
import pandas as pd
import numpy as np
import nltk, re, json

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.datasets as transforms

from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
# store train sentences 
train_file = 'data/train'
dev_file = 'data/dev'
test_file = 'data/test'
dummy_file ='data/dummy'

In [30]:
# read train/test file, each line as {s_idx, word, tag} tuple, store in a list
def readFile(file):
    f = open(file)
    lines = f.readlines()
    words = []
    for line in lines:
        if line.strip():
            words.append(line.strip().split(' '))
    return words

In [9]:
# DF: index - s_idx - word - tag
train_lines = readFile(train_file)
df = pd.DataFrame(train_lines, columns = ["s_idx", "word", "tag"])
df.head(5)

,s_idx,word,tag
0,1,EU,B-ORG
1,2,rejects,O
2,3,German,B-MISC
3,4,call,O
4,5,to,O


In [10]:
# Randomly select some rare words to be <unk> words
unique_words = df["word"].value_counts().reset_index()
unique_words.columns = ["word", "freq"]
threshold = 1
# words with freq > 1
vocab_words = unique_words[ unique_words['freq'] > threshold ]
# words with freq = 1
rare_words = unique_words[ unique_words['freq'] == threshold ]

print("vocab words:", vocab_words.shape[0])
print("rare words:", rare_words.shape[0])

vocab words: 11983
rare words: 11641


In [23]:
# Randomly select 3000 words from rare words to set as unknown words
unk_count = 3000
unk_words = rare_words.sample(unk_count)

# drop the selected rare words from vocab
rare_words = rare_words.drop(unk_words.index)

# build new vocab = freq_words + rest_rare_words + <unk>
vocab_words = vocab_words.append(rare_words, ignore_index=True)

# custom words unk, pad etc
custom_vocab = ['<unk>']

# main vocab list, to generate embedding
vocab_set = set(custom_vocab + vocab_words['word'].unique().tolist())
vocab_size = len(vocab_set)

In [93]:
# all the vocab
word_to_idx = {word:i for i, word in enumerate(vocab_set)}

# all the unique tags
unique_tags = set(df["tag"].unique())
tag_to_idx = {tag:i for i, tag in enumerate(unique_tags)}
idx_to_tag = {i:tag for i, tag in enumerate(unique_tags)}

In [50]:
# read files, group words by sentence, return list of sentences
def readData(file):
    f = open(file)
    lines = f.readlines()
    sentences = []
    sentence = []
    for line in lines:
        if not line.strip():
            sentences.append(sentence.copy())
            sentence.clear()
        else:
            sentence.append(line.strip().split(' '))
    # append the last sentence
    sentences.append(sentence.copy())
    return sentences

In [51]:
# word = [idx, word, tag]  train_data = list of sentences in term of list of words
train_data = readData(train_file)

dev_data = readData(dev_file)
# word = [idx, word]
test_data = readData(test_file)

# Dummy test data
dummy_file ='data/dummy'
dummy_data = readData(dummy_file)

In [183]:
print(len(train_sentences), len(dev_sentences), len(test_sentences))

14987 3466 3684


In [58]:
dummy_data

[[['1', 'Weaver', 'B-PER'],
  ['2', 'shot', 'O'],
  ['3', 'to', 'O'],
  ['4', 'prominence', 'O'],
  ['5', 'in', 'O'],
  ['6', '1994', 'O'],
  ['7', 'when', 'O'],
  ['8', 'he', 'O'],
  ['9', 'won', 'O'],
  ['10', 'the', 'O'],
  ['11', 'English', 'B-MISC'],
  ['12', '2,000', 'B-MISC'],
  ['13', 'Guineas', 'I-MISC'],
  ['14', 'on', 'O'],
  ['15', 'Mister', 'B-LOC'],
  ['16', 'Baileys', 'I-LOC'],
  ['17', 'in', 'O'],
  ['18', 'his', 'O'],
  ['19', 'first', 'O'],
  ['20', 'ride', 'O'],
  ['21', 'in', 'O'],
  ['22', 'a', 'O'],
  ['23', 'classic', 'O'],
  ['24', '.', 'O']],
 [['1', 'Results', 'O'],
  ['2', 'of', 'O'],
  ['3', 'English', 'B-MISC'],
  ['4', 'league', 'O'],
  ['5', 'matches', 'O']]]

In [70]:
# Preapare training data
def processData(tuples):
    training_data = []
    for t in tuples:
        training_data.append( ( [ word[1] if word[1] in word_to_idx else '<unk>' for word in t ], [ word[2] for word in t ] ) )
    return training_data

# Convert sequence into tensor
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [68]:
training_data = processData(dummy_sentences)

In [78]:
for sent, t in training_data:
    print(sent)
    print(t)
    print("---")

['Weaver', 'shot', 'to', 'prominence', 'in', '1994', 'when', 'he', 'won', 'the', 'English', '2,000', 'Guineas', 'on', 'Mister', 'Baileys', 'in', 'his', 'first', 'ride', 'in', 'a', '<unk>', '.']
['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'B-MISC', 'I-MISC', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
---
['Results', 'of', 'English', 'league', 'matches']
['O', 'O', 'B-MISC', 'O', 'O']
---


In [65]:
embedding_dim = 100
hidden_dim = 256
vocab_size = len(word_to_idx)
tagset_size = len(tag_to_idx)

lstm_layer = 1
lstm_dropout = 0.33
linear_out_dim = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [63]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
    
    # sentence [seq, batch, embed_dim]
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [98]:
model = LSTMTagger(embedding_dim, hidden_dim, vocab_size, tagset_size)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [99]:
# Before training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_idx)
    tag_scores = model(inputs)
    print([idx_to_tag[i] for i in torch.argmax(tag_scores, dim=1).tolist()])

print_every = 30
    
for epoch in range(300):
    for sentence, tags in training_data:
        model.zero_grad()
        
        sentence_in = prepare_sequence(sentence, word_to_idx)
        targets = prepare_sequence(tags, tag_to_idx)
        
        tag_scores = model(sentence_in)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
    if epoch%print_every == 0:
        print(loss)
    
    
# After training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_idx)
    tag_scores = model(inputs)
    print([idx_to_tag[i] for i in torch.argmax(tag_scores, dim=1).tolist()])

['I-LOC', 'I-ORG', 'B-MISC', 'B-MISC', 'B-MISC']
tensor(2.1384, grad_fn=<NllLossBackward>)
tensor(0.0834, grad_fn=<NllLossBackward>)
tensor(0.0247, grad_fn=<NllLossBackward>)
tensor(0.0124, grad_fn=<NllLossBackward>)
tensor(0.0076, grad_fn=<NllLossBackward>)
tensor(0.0053, grad_fn=<NllLossBackward>)
tensor(0.0040, grad_fn=<NllLossBackward>)
tensor(0.0031, grad_fn=<NllLossBackward>)
tensor(0.0026, grad_fn=<NllLossBackward>)
tensor(0.0022, grad_fn=<NllLossBackward>)
['O', 'O', 'B-MISC', 'O', 'O']


In [100]:
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_idx)
    tag_scores = model(inputs)
    print([idx_to_tag[i] for i in torch.argmax(tag_scores, dim=1).tolist()])

['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'B-MISC', 'I-MISC', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [102]:
training_data[0][1]

['B-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-MISC',
 'B-MISC',
 'I-MISC',
 'O',
 'B-LOC',
 'I-LOC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [389]:
class BLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes, linear_out_dim, lstm_layer, lstm_dropout):
        super(BLSTM, self).__init__()
        self.hidden_size = hidden_dim
        self.num_layers = lstm_layer
        self.out_size = num_classes
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.bilstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=self.hidden_size,
            batch_first=True,
            bidirectional=True)
        # Linear 1
        self.fc1 = nn.Linear(2*self.hidden_size,linear_out_dim)
        # classifier linear
        self.fc2 = nn.Linear(linear_out_dim, num_classes)
        self.dropout = nn.Dropout(lstm_dropout)
        
        
        
    def forward(self, x):
        batch_size, seq_len = x.shape
        
#         h0 = torch.randn(self.num_layers*2, batch_size, self.hidden_size).to(device)
#         c0 = torch.randn(self.num_layers*2, batch_size, self.hidden_size).to(device)
        h0 = torch.randn(self.num_layers*2, batch_size, self.hidden_size).to(device)
        c0 = torch.randn(self.num_layers*2, batch_size, self.hidden_size).to(device)
        # embedding layer + dropout
        x = self.dropout(self.embedding(x))
        # BLSTM layer
        x , _ = self.bilstm(x)
        # Linear + ELU
        x = F.elu(self.fc1(x))
        # classifier
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x.view(seq_len, self.out_size)
    

In [329]:

input = torch.randn(2,10)
out = F.elu(input)
print(out)
fc = nn.Linear(10,5)
out = fc(out)
print(out)
F.augmaF.softmax(out, dim=1)）

tensor([[ 1.1715,  0.3064, -0.1101,  0.5220, -0.4690, -0.6432, -0.3959, -0.0424,
          1.0491, -0.0986],
        [ 0.9205,  0.5053, -0.2874, -0.0737, -0.8828, -0.5819,  0.3481,  1.1651,
          0.3174,  0.9363]])
tensor([[-0.3601, -1.0359,  0.1784,  0.9853,  0.2091],
        [ 0.0415, -0.8341,  0.7340,  0.5089, -0.0962]],
       grad_fn=<AddmmBackward>)


tensor([[0.1133, 0.0576, 0.1941, 0.4349, 0.2001],
        [0.1700, 0.0708, 0.3398, 0.2713, 0.1481]], grad_fn=<SoftmaxBackward>)

In [ ]:
blstm_model = BLSTM(len(vocab_set),embedding_dim, hidden_dim, num_classes, linear_out_dim, lstm_layer, lstm_dropout).to(device)
optimizer = torch.optim.SGD(blstm_model.parameters(), lr=0.1)
loss_func = nn.CrossEntropyLoss().to(device)
epochs = 50

input_batch, target_batch = makeData(train_sentences)

for epoch in range(epochs):
    # train one sentence at a time
    for input, target in tqdm(zip(input_batch, target_batch), total=len(input_batch)):

        x = torch.LongTensor([input]).to(device)
        y = torch.LongTensor(target).to(device)
        preds = blstm_model(x)
        loss = loss_func(preds,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(loss)

    

100%|██████████| 14987/14987 [00:42<00:00, 353.68it/s]


tensor(2.1972, device='cuda:0', grad_fn=<NllLossBackward>)


100%|██████████| 14987/14987 [00:43<00:00, 344.11it/s]


tensor(2.1972, device='cuda:0', grad_fn=<NllLossBackward>)


100%|██████████| 14987/14987 [00:41<00:00, 357.13it/s]


...

100%|██████████| 14987/14987 [00:42<00:00, 356.20it/s]


tensor(2.1972, device='cuda:0', grad_fn=<NllLossBackward>)


100%|██████████| 14987/14987 [00:42<00:00, 349.44it/s]


tensor(2.1972, device='cuda:0', grad_fn=<NllLossBackward>)


 91%|█████████ | 13662/14987 [00:38<00:03, 360.10it/s]

In [384]:
blstm_model = BLSTM(len(vocab_set),embedding_dim, hidden_dim, num_classes, linear_out_dim, lstm_layer, lstm_dropout).to(device)
blstm_model

BLSTM(
  (embedding): Embedding(20625, 100)
  (bilstm): LSTM(100, 256, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=512, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=9, bias=True)
  (dropout): Dropout(p=0.33, inplace=False)
)